In [1]:
import psycopg2
import os
import pandas as pd
from dotenv import load_dotenv
load_dotenv()

True

In [9]:
# Connect to Redshift
conn = psycopg2.connect(
    dbname=os.getenv('REDSHIFT_SCHEMA'),
    user=os.getenv('REDSHIFT_USER'),
    password=os.getenv('REDSHIFT_PASSWORD'),
    host=os.getenv('REDSHIFT_HOST'),
    port=5439
)

cur = conn.cursor()

# Example schema — adjust columns to match your CSV
create_table_sql = """
DROP TABLE IF EXISTS DAILY_CRIPTO_PRICES;

CREATE TABLE DAILY_CRIPTO_PRICES (
    usd FLOAT,
    cripto VARCHAR(100),
    date TIMESTAMP
);
"""

cur.execute(create_table_sql)
conn.commit()

#cur.close()

In [10]:
df = pd.read_sql("SELECT * FROM DAILY_CRIPTO_PRICES;", conn)
print(df.head())

/var/folders/1k/sh8hpzz512gfjrxqfmzwshbc0000gn/T/ipykernel_6506/2560394726.py:1: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql("SELECT * FROM DAILY_CRIPTO_PRICES;", conn)


Empty DataFrame
Columns: [usd, cripto, date]
Index: []


In [6]:
prices_historic=pd.DataFrame()
for currency in ["bitcoin","ethereum","tether","solana"]:
    currency_historic = pd.read_csv(f'~/Downloads/{currency}-usd-max.csv',
    dtype={"total_volume": "float64", "price": "float64","market_cap":"float64"},
    parse_dates=["snapped_at"],
    date_parser=lambda x: pd.to_datetime(x, utc=True).strftime("%d-%m-%Y"))
    currency_historic.columns = ['date', 'usd', 'market_cap', 'total_volume']
    currency_historic = currency_historic[['date', 'usd']]
    currency_historic['cripto'] = currency
    prices_historic = pd.concat([prices_historic,currency_historic],ignore_index=True)

/var/folders/1k/sh8hpzz512gfjrxqfmzwshbc0000gn/T/ipykernel_6506/3158066530.py:3: FutureWarning: The argument 'date_parser' is deprecated and will be removed in a future version. Please use 'date_format' instead, or read your data in as 'object' dtype and then call 'to_datetime'.
  currency_historic = pd.read_csv(f'~/Downloads/{currency}-usd-max.csv',
/var/folders/1k/sh8hpzz512gfjrxqfmzwshbc0000gn/T/ipykernel_6506/3158066530.py:3: UserWarning: Parsing dates in %d-%m-%Y format when dayfirst=False (the default) was specified. Pass `dayfirst=True` or specify a format to silence this warning.
  currency_historic = pd.read_csv(f'~/Downloads/{currency}-usd-max.csv',
/var/folders/1k/sh8hpzz512gfjrxqfmzwshbc0000gn/T/ipykernel_6506/3158066530.py:3: FutureWarning: The argument 'date_parser' is deprecated and will be removed in a future version. Please use 'date_format' instead, or read your data in as 'object' dtype and then call 'to_datetime'.
  currency_historic = pd.read_csv(f'~/Downloads/{cur

In [7]:
prices_historic['date'] = prices_historic['date'].astype('datetime64[ns]')

In [8]:
prices_historic[prices_historic['date']>='2023-01-01']

,date,usd,cripto
3533,2023-01-01,16540.693625,bitcoin
3534,2023-01-02,16615.969318,bitcoin
3535,2023-01-03,16674.342536,bitcoin
3536,2023-01-04,16677.646186,bitcoin
3537,2023-01-05,16855.174739,bitcoin
...,...,...,...
14262,2025-09-11,158.156189,solana
14263,2025-10-11,164.268421,solana
14264,2025-11-11,167.548409,solana
14265,2025-12-11,154.346288,solana


In [ ]:
from sqlalchemy import create_engine, text

engine = create_engine(os.getenv("REDSHIFT"))

# Insert a single row
with engine.connect() as conn:
    for index, row in prices_historic[prices_historic['date']>='2023-01-01'].iterrows():
        conn.execute(
        text("INSERT INTO DAILY_CRIPTO_PRICES (usd, cripto,date) VALUES (:usd, :cripto,:date)"),
        {"usd": row.usd, "cripto":row.cripto,'date':row.date}
        )

/var/folders/1k/sh8hpzz512gfjrxqfmzwshbc0000gn/T/ipykernel_6506/15522656.py:8: RemovedIn20Warning: Deprecated API features detected! These feature(s) are not compatible with SQLAlchemy 2.0. To prevent incompatible upgrades prior to updating applications, ensure requirements files are pinned to "sqlalchemy<2.0". Set environment variable SQLALCHEMY_WARN_20=1 to show all deprecation warnings.  Set environment variable SQLALCHEMY_SILENCE_UBER_WARNING=1 to silence this message. (Background on SQLAlchemy 2.0 at: https://sqlalche.me/e/b8d9)
  conn.execute(


In [2]:
conn = psycopg2.connect(
    dbname=os.getenv('REDSHIFT_SCHEMA'),
    user=os.getenv('REDSHIFT_USER'),
    password=os.getenv('REDSHIFT_PASSWORD'),
    host=os.getenv('REDSHIFT_HOST'),
    port=5439
)
df = pd.read_sql("SELECT * FROM DAILY_CRIPTO_PRICES;", conn)
df

/var/folders/1k/sh8hpzz512gfjrxqfmzwshbc0000gn/T/ipykernel_7304/2335844845.py:8: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql("SELECT * FROM DAILY_CRIPTO_PRICES;", conn)


,usd,cripto,date
0,16540.693625,bitcoin,2023-01-01
1,16615.969318,bitcoin,2023-01-02
2,16674.342536,bitcoin,2023-01-03
3,16677.646186,bitcoin,2023-01-04
4,16855.174739,bitcoin,2023-01-05
...,...,...,...
411,51958.230291,bitcoin,2024-02-16
412,52166.434310,bitcoin,2024-02-17
413,51684.517493,bitcoin,2024-02-18
414,52138.485789,bitcoin,2024-02-19
